In [1]:
HDFStore=pd.HDFStore

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_docs as tfdocs
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from utils import config


In [2]:
# loading data parameters
num_coefficients=int(config["coefficients"])
root_path=config["root_path"]
hw_results_path= config["hw_results_path"]

# training parameters
seed=int(config["seed"]) if "seed" in config.keys() else 42

dr=float(config["dropout"]) if "dropout" in config.keys() else 0.2
lr2=float(config["lr2"]) if "lr2" in config.keys() else 1e-3
lr1=float(config["lr1"]) if "lr1" in config.keys() else 1e-4
lr=float(config["lr"]) if "lr" in config.keys() else 8e-4

num_epochs=int(config["num_epochs"]) if "num_epochs" in config.keys() else 1000
num_features=int(config["features"]) if "features" in config.keys() else 4096
mini_batch_size=int(config["mini_batch_size"]) if "mini_batch_size" in config.keys() else 4

main_units=int(config["main_units"]) if "main_units" in config.keys() else 64
secondary_units=int(config["secondary_units"]) if "secondary_units" in config.keys() else 16
last_unit=int(config["last_unit"]) if "last_unit" in config.keys() else 8
lstm_units=int(config["lstm_units"]) if "lstm_units" in config.keys() else 64
num_classes=int(config["num_classes"]) if "num_classes" in config.keys() else 1


print_sample=False

In [3]:
#Load data from hdf5 file
rdo_root_path = path.join(root_path,hw_results_path)
h5file = path.join(rdo_root_path, "archimedean-")
h5filename = h5file + str(num_coefficients) + "-splits.h5"
print(h5filename)
hdf = HDFStore(h5filename)
hdf.keys()

/data/elekin/data/results/03-HANDWRITTING/archimedean-17-splits.h5


['/residues/train/features',
 '/residues/train/labels',
 '/residues/test/features',
 '/residues/test/labels']

In [4]:
#Load and scale timeseries between 0 and 1
x_train = hdf["/residues/train/features"].values.astype('float32')
y_train = hdf["/residues/train/labels"].values.astype('int8').reshape(-1,1)
x_test = hdf["/residues/test/features"].values.astype('float32')
y_test = hdf["/residues/test/labels"].values.astype('int8').reshape(-1,1)
hdf.close()

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).take(len(x_train)).batch(mini_batch_size).prefetch(2).cache()
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).take(len(x_test)).batch(mini_batch_size).prefetch(2).cache()
steps_per_epoch = round(len(train_dataset)/mini_batch_size)

if print_sample:
    for feat, targ in test_dataset.take(10):
        print ('Features test: {}, Target: {}'.format(feat, targ))

    for feat, targ in test_dataset.take(10):
        print ('Features train: {}, Target: {}'.format(feat, targ))

print("{0} train batches and {1} test batches of {2} mini batch size and {3} steps per epoch".format(len(train_dataset), 
                                                                              len(test_dataset),
                                                                              mini_batch_size,
                                                                                steps_per_epoch))
test_dataset

9 train batches and 5 test batches of 4 mini batch size and 2 steps per epoch


<CacheDataset shapes: ((None, 4096), (None, 1)), types: (tf.float32, tf.int8)>